In [ ]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host":HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

#### Verificando dados de usuário

In [ ]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

### Buscando hitorico de orders no BD

In [ ]:
# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 20).date()
data_fim = datetime(2023, 12, 23).date()

In [ ]:
# Abra a conexão com o banco de dados
try:
    conn = psycopg2.connect(**db_config)
    
    # Construa a consulta SQL com a condição de data
    sql_query = f"SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    # Execute a consulta e leia os dados em um DataFrame
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

In [ ]:
df_orders.sample()

In [ ]:

print(df_orders['fulfilled'].value_counts())
df_orders.columns

### Buscando dados de produtos no BD

In [ ]:
# Buscando dados de produtos no BD
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM tiny_fulfillment"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    
except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

In [ ]:
df_codes.sample()

In [ ]:
df_codes.sample()

### DF de produtos e orders

In [ ]:
df_filtered = pd.merge(df_orders, df_codes, left_on=['ml_code', 'seller_sku'], right_on=['ml_code', 'ml_sku'], how='left')

df_filtered.sample(3)

In [ ]:
df_filtered.shape

In [ ]:
df_filtered.columns

In [ ]:
print(len(df_filtered['ml_inventory_id'].unique()))
df_filtered['ml_inventory_id'].unique()

In [ ]:
df_filtered['ml_inventory_id'] = df_filtered['ml_inventory_id'].replace('NaN', pd.NA)
df_filtered = df_filtered.dropna(subset=['ml_inventory_id'])


In [ ]:
print(len(df_filtered['ml_inventory_id'].unique()))
df_filtered['ml_inventory_id'].unique()

In [ ]:
df_filtered['payment_status'].value_counts()

In [ ]:
# filtrando apenas aprovados
df_filtered = df_filtered[df_filtered['payment_status'] == 'approved']
df_filtered['payment_status'].value_counts()

In [ ]:
df_filtered.sample(3)

### Calculando quantidade de produtos vendidos

In [ ]:
# Soma de produtos vendidos
soma_por_ml_inventory_id = df_filtered.groupby('ml_inventory_id')['quantity'].sum().reset_index()
print(df_filtered.shape)
print(soma_por_ml_inventory_id.shape)
soma_por_ml_inventory_id.head(3)


In [ ]:
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim}'"
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

In [ ]:
# Ordenando stock por data
df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date
df_stock = df_stock.drop(['created_at'], axis=1)
print(df_stock.shape)
df_stock.head(3)

In [ ]:
df_stock.tail(3)

In [ ]:
# Filtrando dados de orders por datas
df_orders = df_filtered.copy()
df_orders['date_approved'] = pd.to_datetime(df_orders['date_approved'])
df_orders['data'] = df_orders['date_approved'].dt.date
df_orders = df_orders.drop(['date_closed', 'created_at', 'updated_at'], axis=1)
df_orders.head(3)

In [ ]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by='data', ascending=False).reset_index(drop=True)

df_stock.head()

In [ ]:
## Contando dias em que produto esteve disponível 
days_available = df_stock.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
days_available = days_available.rename(columns={'has_stock': 'days_available'})

days_available.sample(5)

In [ ]:
# Unindo DFs
df_stock = df_stock.merge(days_available, on='ml_inventory_id', how='left')
df_stock

In [ ]:
# Total de vendas por ml_inventory_id
total_sales_by_id = df_orders.groupby('ml_inventory_id')['quantity'].sum().reset_index()
total_sales_by_id.sample(5)


In [ ]:
# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(df_orders, total_sales_by_id, on='ml_inventory_id', how='left')

df_total_sales.rename(columns={'quantity_x': 'sales_quantity'}, inplace=True)
df_total_sales.rename(columns={'quantity_y': 'total_sales_quantity'}, inplace=True)
df_total_sales['sales_quantity'] = df_total_sales['sales_quantity'].fillna(0)
df_total_sales['total_sales_quantity'] = df_total_sales['total_sales_quantity'].fillna(0)

df_total_sales.sample()


In [ ]:
df_stock_days_available = df_stock.copy()

indices_recentes = df_stock_days_available.groupby('ml_inventory_id')['data'].idxmax()
indices_recentes


In [ ]:
df_stock_days_available = df_stock_days_available.loc[indices_recentes]
df_stock_days_available = df_stock_days_available.drop(['has_stock'], axis=1)

df_stock_days_available.sample(3)


In [ ]:
df_stock_days_available.columns

In [ ]:
df_total_sales_filter = df_total_sales[['ml_inventory_id', 'ml_code','ml_sku', 'tiny_id', 'date_approved', 'sales_quantity', 'total_sales_quantity']]
print(df_total_sales_filter.shape)
df_total_sales_filter = df_total_sales_filter.drop_duplicates()
print(df_total_sales_filter.shape)
df_total_sales_filter.sample(3)

In [ ]:
# _df_orders_filtrado_ = df_orders_filtrado_.copy()

indices_recentes = df_total_sales.groupby('ml_inventory_id')['date_approved'].idxmax()
indices_recentes

df_total_sales = df_total_sales.loc[indices_recentes]
df_total_sales = df_total_sales.drop(['date_approved','sales_quantity'], axis=1)

df_total_sales.sample()

In [ ]:
print(df_total_sales.shape)
print(df_stock_days_available.shape)

In [ ]:
df_stock_days_available.sample()

## Juntando dias disponiveis e total de vendas

In [ ]:
df_to_calc = pd.merge(df_stock_days_available, df_total_sales, on='ml_inventory_id', how='left')
df_to_calc['total_sales_quantity'] = df_to_calc['total_sales_quantity'].fillna(0)
df_to_calc.shape

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows')

df_to_calc = df_to_calc.dropna(subset=['ml_code', 'ml_sku'])
print(df_to_calc.shape)
df_to_calc.sample()

In [ ]:
df_to_calc['total_sales_quantity'] = df_to_calc['total_sales_quantity'].fillna(0)
df_to_calc['days_available'] = df_to_calc['days_available'].fillna(0)
df_to_calc.shape


### Calculando métricas

In [ ]:
df_to_calc['media_prod_days_available'] = (df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])
df_to_calc['media_prod_days_available'] = df_to_calc['media_prod_days_available'].fillna(0)


In [ ]:
df_to_calc['media_prod_days_available'] = (df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])
df_to_calc['media_prod_days_available'] = df_to_calc['media_prod_days_available'].fillna(0)

days = 30


df_to_calc['period_send_fulfillment'] = np.ceil((df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])* days - df_to_calc['available_quantity'])
df_to_calc['period_send_fulfillment'] = df_to_calc['period_send_fulfillment'].fillna(0)

In [ ]:
df_to_calc['today_send_fulfillment'] = np.ceil((df_to_calc['total_sales_quantity'] / df_to_calc['days_available']) - df_to_calc['available_quantity'])
df_to_calc['today_send_fulfillment'] = df_to_calc['today_send_fulfillment'].fillna(0)

In [ ]:
# x = df_to_calc.copy()

In [ ]:
df_to_calc = df_to_calc.drop(['category_id','pack_id','title','order_status', 'payment_status', 'seller_sku', 'shipping_id', 'fulfilled', 'mcenter_id', 'variation_id', 'var_code', 'ad_title', 'data_y', 'tiny_sku'], axis=1)
df_to_calc = df_to_calc.rename(columns={'data_x':'data'})


In [ ]:
df_to_calc.sample()

In [ ]:
df_fulfillment_greater_zero = df_to_calc[df_to_calc['period_send_fulfillment'] > 0]

print(df_fulfillment_greater_zero.shape)
df_fulfillment_greater_zero.sample()

In [ ]:
df_fulfillment_less_one = df_to_calc[df_to_calc['period_send_fulfillment'] <= 0]
df_fulfillment_less_one.sample()

### Comparando com a Tiny

In [ ]:
# Pegando dados de stock da tiny
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM tiny_stock_hist"
    df_tiny_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

In [ ]:
# Obtém a data de hoje
data_de_hoje = pd.to_datetime('today').date()

# Filtra apenas as linhas onde 'created_at' é igual à data de hoje
df_hoje = df_tiny_stock[df_tiny_stock['created_at'].dt.date == data_de_hoje]
df_hoje.shape

In [ ]:
df_resultado = df_hoje[df_hoje['tiny_id'].isin(df_fulfillment_greater_zero['tiny_id'])]
print(df_resultado.shape)
print(df_fulfillment_greater_zero.shape)

In [ ]:
df_resultado

In [ ]:
x = df_resultado[df_resultado['tiny_id']=='698412654']
x

In [ ]:
df_fulfillment_greater_zero.sample()

In [ ]:
df_resultado_ = pd.merge(df_fulfillment_greater_zero, df_resultado, on='tiny_id', how='left')
df_resultado_.shape

In [ ]:
df_resultado_

In [ ]:
df_send = df_resultado_[df_resultado_['deposito_saldo'] > 0]

cols = ['ml_inventory_id', 'ml_code','ml_sku', 'tiny_id',  'days_available', 'total_sales_quantity', 'media_prod_days_available', 'period_send_fulfillment', 'today_send_fulfillment', 'saldo_reservado', 'deposito_nome', 'deposito_saldo', 'deposito_empresa', 'data']
df_send = df_send[cols]
df_send

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host":HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Defina as datas de início e fim desejadas
data_inicio = datetime(2023, 11, 20).date()
data_fim = datetime(2023, 12, 23).date()

# Registra o tempo antes da execução
start_prog = time.time()

# Abra a conexão com o banco de dados
try:
    conn = psycopg2.connect(**db_config)
    
    # Construa a consulta SQL com a condição de data
    sql_query = f"SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '{data_inicio}' AND '{data_fim}'"
    print(sql_query)
    # Execute a consulta e leia os dados em um DataFrame
    df_orders = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro do psycopg2 ao consultar ml_orders_hist: {e}")

except Exception as e:
    print(f"Erro ao consultar ml_orders_hist: {e}")
    # logger.error(f"Erro ao consultar ml_orders_hist: {e}")

finally:
    if conn is not None:
        conn.close()

# filtrando apenas orders do fulfillment
df_orders = df_orders[df_orders['fulfilled'] == True]

# Buscando dados de produtos no BD
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM tiny_fulfillment"
    df_codes = pd.read_sql(sql_query, conn)
except psycopg2.Error as e:
    # logger.error(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")
    
except Exception as e:
    # logger.error(f"Erro ao consultar tabela tiny_fulfillment: {e}")
    print(f"Erro ao consultar tabela tiny_fulfillment: {e}")

finally:
    if conn is not None:
        conn.close()

df_codes['ml_code'] = df_codes['ml_code'].apply(lambda x: 'MLB' + str(x))

# unindo DFs
df_filtered = pd.merge(df_orders, df_codes, left_on=['ml_code', 'seller_sku'], right_on=['ml_code', 'ml_sku'], how='left')
df_filtered['ml_inventory_id'] = df_filtered['ml_inventory_id'].replace('NaN', pd.NA)
df_filtered = df_filtered.dropna(subset=['ml_inventory_id'])

# filtrando apenas aprovados
df_filtered = df_filtered[df_filtered['payment_status'] == 'approved']

# Soma de produtos vendidos
soma_por_ml_inventory_id = df_filtered.groupby('ml_inventory_id')['quantity'].sum().reset_index()

# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = f"SELECT * FROM fulfillment_stock_hist WHERE created_at BETWEEN '{data_inicio}' AND '{data_fim}'"
    df_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()
        
# Ordenando stock por data
df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date
df_stock = df_stock.drop(['created_at'], axis=1)

# Filtrando dados de orders por datas
df_orders = df_filtered.copy()
df_orders['date_approved'] = pd.to_datetime(df_orders['date_approved'])
df_orders['data'] = df_orders['date_approved'].dt.date
df_orders = df_orders.drop(['date_closed', 'created_at', 'updated_at'], axis=1)

## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock = df_stock.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock = df_stock.sort_values(by='data', ascending=False).reset_index(drop=True)

## Contando dias em que produto esteve disponível 
days_available = df_stock.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
days_available = days_available.rename(columns={'has_stock': 'days_available'})

# Unindo DFs
df_stock = df_stock.merge(days_available, on='ml_inventory_id', how='left')

# Total de vendas por ml_inventory_id
total_sales_by_id = df_orders.groupby('ml_inventory_id')['quantity'].sum().reset_index()

# Acrescentando total de vendas ao DF
df_total_sales = pd.merge(df_orders, total_sales_by_id, on='ml_inventory_id', how='left')

df_total_sales.rename(columns={'quantity_x': 'sales_quantity'}, inplace=True)
df_total_sales.rename(columns={'quantity_y': 'total_sales_quantity'}, inplace=True)
df_total_sales['sales_quantity'] = df_total_sales['sales_quantity'].fillna(0)
df_total_sales['total_sales_quantity'] = df_total_sales['total_sales_quantity'].fillna(0)

df_stock_days_available = df_stock.copy()

indices_recentes = df_stock_days_available.groupby('ml_inventory_id')['data'].idxmax()

df_stock_days_available = df_stock_days_available.loc[indices_recentes]
df_stock_days_available = df_stock_days_available.drop(['has_stock'], axis=1)

df_total_sales_filter = df_total_sales[['ml_inventory_id', 'ml_code','ml_sku', 'tiny_id', 'date_approved', 'sales_quantity', 'total_sales_quantity']]
df_total_sales_filter = df_total_sales_filter.drop_duplicates()

indices_recentes = df_total_sales.groupby('ml_inventory_id')['date_approved'].idxmax()
df_total_sales = df_total_sales.loc[indices_recentes]
df_total_sales = df_total_sales.drop(['date_approved','sales_quantity'], axis=1)

## Juntando dias disponiveis e total de vendas
df_to_calc = pd.merge(df_stock_days_available, df_total_sales, on='ml_inventory_id', how='left')
df_to_calc['total_sales_quantity'] = df_to_calc['total_sales_quantity'].fillna(0)

df_to_calc = df_to_calc.dropna(subset=['ml_code', 'ml_sku'])
df_to_calc['total_sales_quantity'] = df_to_calc['total_sales_quantity'].fillna(0)
df_to_calc['days_available'] = df_to_calc['days_available'].fillna(0)

# Calculando Métricas
df_to_calc['media_prod_days_available'] = (df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])
df_to_calc['media_prod_days_available'] = df_to_calc['media_prod_days_available'].fillna(0)

df_to_calc['media_prod_days_available'] = (df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])
df_to_calc['media_prod_days_available'] = df_to_calc['media_prod_days_available'].fillna(0)

days = 30


df_to_calc['period_send_fulfillment'] = np.ceil((df_to_calc['total_sales_quantity'] / df_to_calc['days_available'])* days - df_to_calc['available_quantity'])
df_to_calc['period_send_fulfillment'] = df_to_calc['period_send_fulfillment'].fillna(0)

df_to_calc['today_send_fulfillment'] = np.ceil((df_to_calc['total_sales_quantity'] / df_to_calc['days_available']) - df_to_calc['available_quantity'])
df_to_calc['today_send_fulfillment'] = df_to_calc['today_send_fulfillment'].fillna(0)

# Selecionando a exibição
df_to_calc = df_to_calc.drop(['category_id','pack_id','title','order_status', 'payment_status', 'seller_sku', 'shipping_id', 'fulfilled', 'mcenter_id', 'variation_id', 'var_code', 'ad_title', 'data_y', 'tiny_sku'], axis=1)
df_to_calc = df_to_calc.rename(columns={'data_x':'data'})


## Filtrando produtos a serem envidos ou não, period_send_fulfillment > 0 produto deve ser enviado 
df_fulfillment_greater_zero = df_to_calc[df_to_calc['period_send_fulfillment'] > 0]
df_fulfillment_less_one = df_to_calc[df_to_calc['period_send_fulfillment'] <= 0]


## Comparando com stock Tiny

# Pegando dados de stock da tiny
# Buscando histórico de estoque na tabela
try:
    conn = psycopg2.connect(**db_config)

    sql_query = "SELECT * FROM tiny_stock_hist"
    df_tiny_stock = pd.read_sql(sql_query, conn)

except psycopg2.Error as e:
    print(f"Erro do psycopg2 ao consultar fulfillment_stock: {e}")

except Exception as e:
    print(f"Erro ao consultar fulfillment_stock: {e}")

finally:
    if conn is not None:
        conn.close()

# Obtém a data de hoje
data_de_hoje = pd.to_datetime('today').date()

# Filtra apenas as linhas onde 'created_at' é igual à data de hoje
df_today = df_tiny_stock[df_tiny_stock['created_at'].dt.date == data_de_hoje]

df_res = df_today[df_today['tiny_id'].isin(df_fulfillment_greater_zero['tiny_id'])]

df_res_ = pd.merge(df_fulfillment_greater_zero, df_res, on='tiny_id', how='left')

df_send = df_res_[df_res_['deposito_saldo'] > 0]

cols = ['ml_inventory_id', 'ml_code','ml_sku', 'tiny_id',  'days_available', 'total_sales_quantity', 'media_prod_days_available', 'period_send_fulfillment', 'today_send_fulfillment', 'saldo_reservado', 'deposito_nome', 'deposito_saldo', 'deposito_empresa', 'data']
df_send = df_send[cols]


SELECT * FROM ml_orders_hist WHERE date_closed BETWEEN '2023-11-20' AND '2023-12-23'


/tmp/ipykernel_142710/1105817739.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_orders = pd.read_sql(sql_query, conn)
/tmp/ipykernel_142710/1105817739.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)
/tmp/ipykernel_142710/1105817739.py:99: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)
/tmp/ipykernel_142710/1105817739.py:200: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite

In [2]:
df_send

,ml_inventory_id,ml_code,ml_sku,tiny_id,days_available,total_sales_quantity,media_prod_days_available,period_send_fulfillment,today_send_fulfillment,saldo_reservado,deposito_nome,deposito_saldo,deposito_empresa,data
4,AVOX05934,MLB2003190501,FULLBASSOSF80,729131555,3,1.0,0.333333,9.0,-0.0,0.0,Musical Filial,38.0,mcenter168,2023-11-23
8,BLHH53768,MLB2004836143,FULLBASSOVTSL52,729127557,3,1.0,0.333333,6.0,-3.0,1.0,M A Bueno,27.0,mcenter168,2023-11-23
11,BLHH53768,MLB2004836143,FULLBASSOVTSL52,729127557,3,1.0,0.333333,6.0,-3.0,1.0,Musical Filial,72.0,mcenter168,2023-11-23
19,BOWK48319,MLB1461189878,FULLKALANI15420,655422932,3,1.0,0.333333,3.0,-6.0,0.0,Musical Matriz,21.0,mcenter168,2023-11-23
24,CLMF99895,MLB1992541482,FULLSG6687,730228363,3,4.0,1.333333,31.0,-7.0,1.0,Mundo das Teclas,590.0,mcenter168,2023-11-23
30,DSGP06967,MLB1992567302,FULLNI5BM,735947207,3,2.0,0.666667,14.0,-5.0,171.0,M A Bueno,977.0,mcenter168,2023-11-23
34,DSGP06967,MLB1992567302,FULLNI5BM,735947207,3,2.0,0.666667,14.0,-5.0,171.0,Musical Matriz,1465.0,mcenter168,2023-11-23
37,DSGP06979,MLB1992567302,FULLNI7AM,747848159,3,1.0,0.333333,3.0,-6.0,157.0,M A Bueno,803.0,mcenter168,2023-11-23
41,DSGP06979,MLB1992567302,FULLNI7AM,747848159,3,1.0,0.333333,3.0,-6.0,157.0,Musical Matriz,2511.0,mcenter168,2023-11-23
44,EIPZ51427,MLB2770292345,FULLVANDOREN2SR2115,748837437,3,5.0,1.666667,42.0,-6.0,100.0,M A Bueno,590.0,mcenter168,2023-11-23
